In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.insert(0, '..')

import os
import definition
from utility import Preprocessor, Embedding
import pandas as pd
import numpy as np

## Data Loading, Preprocessing, and Selection

In [2]:
df = pd.read_csv(definition.DATA_RAW_FILE)
df = df.drop_duplicates('review_id', 'first').reset_index().drop('index', axis=1)
print(len(df))
df.head()

144353


,review_id,review_text,review_source,review_score,category
0,73a965356fe43aa02b113b684f1a365c,Kotor berdebu. Saya tdk berhenti bersin ketika...,befc88fe53f24cb70ce7f9d2efd89858,5.5,kebersihan
1,35f506b353ff95aed86131fb01fb43bc,oke cuma air wastafel berwarna keruh mohon di...,4c5d06b02c97731aaa976179c62dcf76,3.0,fasilitas
2,e93324022b758ce67710708ac115cd18,kamar ada semutnya. kamar mandi bermasalah. bu...,4c5d06b02c97731aaa976179c62dcf76,2.0,kebersihan
3,dff13be030a1a10741848a2d9622cee4,"Kamar mandi bau, airnya bau",4c5d06b02c97731aaa976179c62dcf76,1.0,kebersihan
4,beeff65d3c2e32934261586493934f99,"kamarnya bersih dan nyaman, tetapi perlu di pe...",4c5d06b02c97731aaa976179c62dcf76,3.0,fasilitas


In [3]:
sampling_percentage = 0.6

index = np.arange(len(df))
np.random.shuffle(index)
index_train = index[:int(sampling_percentage * len(df))]

corpus = df['review_text'].values[index_train]
corpus = corpus.astype(str)

In [4]:
prep_corpus = []
for review in corpus:
    prep_corpus.append(Preprocessor.tokenize(review))
print(len(prep_corpus))

86611


## Embedding Construction

In [5]:
size = 50
min_count = 5
min_n = 3
max_n = 3
iter_count = 10

In [6]:
embedding = Embedding(min_count=min_count, size=size, min_n=min_n, max_n=max_n, iter=iter_count) #FastText
# embedding = Embedding(fast_text=False, min_count=min_count, size=size, iter=iter_count) #Word2Vec
embedding.build_vocab(prep_corpus)
embedding.train(prep_corpus, verbose=True)

Epoch #10/10...


In [7]:
embedding.model

In [8]:
embedding_filename = "fasttext_{}.bin".format(size)

embedding.save(os.path.join(definition.MODEL_UTILITY, embedding_filename))

In [9]:
emb = Embedding()
emb.load(os.path.join(definition.MODEL_UTILITY, "word2vec_50.bin"))

In [10]:
emb.get_vector_word("service")

array([-1.7022049e+00, -2.0602486e+00,  9.9179786e-01, -1.7458515e+00,
        9.1210834e-04,  7.1764582e-01,  1.7087953e+00, -6.0669923e-01,
       -1.0208499e-01,  3.0401435e-01, -8.1829858e-01, -2.7640522e-01,
       -3.6003739e-01, -8.8371724e-01,  7.2289541e-02,  2.5206330e-01,
       -5.9040797e-01, -9.2085946e-01,  1.2441402e-01,  1.0682888e+00,
        1.3036884e+00,  3.2723479e+00,  7.6717973e-01,  9.6611863e-01,
        1.4282854e+00,  1.3190644e+00, -2.6156983e-01,  8.5227525e-01,
       -1.3431911e+00,  2.3524494e-01, -2.1572757e+00, -2.0660398e+00,
       -1.0267270e+00,  1.8764496e-01, -1.8040169e+00, -7.3037148e-01,
       -2.0896409e+00, -3.0169935e+00,  1.8700662e+00,  1.0991282e+00,
        2.2655790e+00,  3.0235515e+00,  8.6412990e-01,  1.9274672e+00,
        1.4980316e+00, -8.8511884e-01,  9.5460214e-02, -2.8977257e-01,
       -4.8271167e-01,  1.6822881e+00], dtype=float32)